**Libraries and dependencies**

In [ ]:
!sudo-g5k pip install -U scikit-learn umap-learn plotly xlsx2csv

In [1]:
import time
from datetime import timedelta
import polars as pl
import polars.selectors as cs

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler

import umap.umap_ as umap
from umap import UMAP
import hdbscan

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Read data**

In [3]:
m = pl.read_excel(source="~/Ocean_IA/group_storage/TARA/OM_RGC_V2/Salazar_et_al_2019_Suppl_Info.xlsx",
                  sheet_name="Table_W4",
                  read_csv_options={"null_values": ['NA'], "n_rows": 180})
m = m.rename({"PANGAEA sample id":"PANGAEA.sample.id"})
print(f"Environmental dataframe dimensions: {m.shape}")
 
metaG = pl.read_csv('~/Ocean_IA/group_storage/TARA/OM_RGC_V2/OM-RGC_v2_gene_profile_metaG.tsv.gz',
                    separator='\t',
                    dtypes=([pl.Utf8]+[pl.Float64]*180))
print(f"MetaG dataframe dimensions: {metaG.shape}")

Environmental dataframe dimensions: (180, 38)
MetaG dataframe dimensions: (46775154, 181)


**Tranpose**

In [4]:
t0 = time.time()
s_t = metaG.transpose(include_header=True, column_names=metaG["OMRGC_ID"])
t2= time.time()
print(f"Check tranpose. Time = {str(timedelta(seconds=round(t2-t0)))}")
s_t = s_t.slice(1)
t3=time.time()
print(f"Chek slice. Time = {str(timedelta(seconds=round(t3-t0)))}")
s_t = s_t.cast({cs.starts_with("OM"):pl.Float64})
t4=time.time()
print(f"Check dtype changes. Time = {str(timedelta(seconds=round(t4-t0)))}")
s_t = s_t.rename({"column":"PANGAEA.sample.id"})
t5=time.time()
print(f"Check rename. Time = {str(timedelta(seconds=round(t5-t0)))}")
print("Success. New dataframe dimensions:", s_t.shape)


Check tranpose. Time = 0:19:27
Chek slice. Time = 0:19:50


**Subset without MIX layer**

In [11]:
MIX = m.filter(pl.col("Layer") == "MIX")
print("MIX df dimensions:", MIX.shape)

L = MIX["PANGAEA.sample.id"].to_list()

MIX df dimensions: (7, 38)


In [ ]:
metaG_test = s_t.filter(pl.col("PANGAEA.sample.id") != L)
print("Filtered metaG df dimensions:", metaG_test.shape)

ColumnNotFoundError: unable to find column "PANGAEA.sample.id"; valid columns: ["OMRGC_ID", "TARA_X000000368", "TARA_Y200000002", "TARA_A200000159", "TARA_A200000113", "TARA_X000001036", "TARA_X000000950", "TARA_S200000501", "TARA_A100000164", "TARA_E500000081", "TARA_E500000075", "TARA_E500000331", "TARA_E500000178", "TARA_A100001011", "TARA_A100001015", "TARA_A100001388", "TARA_A100001037", "TARA_A100001035", "TARA_A100001234", "TARA_B100000029", "TARA_B100000003", "TARA_B100000035", "TARA_Y100000022", "TARA_B100000315", "TARA_B100000073", "TARA_Y100000294", "TARA_Y100000287", "TARA_B100000085", "TARA_Y100000031", "TARA_B100000287", "TARA_B100000282", "TARA_B100000131", "TARA_B100000123", "TARA_B100000161", "TARA_B100000242", "TARA_B100000214", "TARA_B100000212", "TARA_B100000378", "TARA_B000000609", "TARA_B000000565", "TARA_B000000557", "TARA_B000000532", "TARA_B100000405", "TARA_B100000408", "TARA_B100000401", "TARA_B000000441", "TARA_B000000460", "TARA_B000000437", "TARA_B00000047

**Normalization**

In [ ]:
t0=time.time()
n = s_t.drop("PANGAEA.sample.id")
N = StandardScaler().fit_transform(n)
t1=time.time()
print(f"Normalization time = {str(timedelta(seconds=round(t1-t0)))}")

In [ ]:
t0=time.time()
umap_model_2D = umap.UMAP(n_neighbors=40, n_components=2, min_dist=0.1, spread=1.0, random_state=42)
umap_2D = umap_model_2D.fit_transform(N)
t1=time.time()
print(f"Process time = {str(timedelta(seconds=round(t1-t0)))}")

In [ ]:
figure_2D = px.scatter(
    umap_2D, x=0, y=1, color=m['Layer'],
    labels={"0": "Dimension 1", "1": "Dimension 2"},
    template="plotly_white", width=1000, height=800, opacity=.6)

figure_2D.update_traces(marker_size=8)
figure_2D.update_layout(legend_itemsizing="constant", legend_font_size=15, font=dict(size=12))
figure_2D.show()

In [ ]:
t0=time.time()
umap_model_3D = umap.UMAP(n_neighbors=40,n_components=3, min_dist=0.1, spread=1.0,random_state=42)
umap_3D = umap_model_3D.fit_transform(N)
t1=time.time()
print(f"Process time = {str(timedelta(seconds=round(t1-t0)))}")

In [ ]:
figure_3D = px.scatter_3d(
    umap_3D, x=0, y=1, z=2, color=m['Layer'],
    labels={"0": "Dimension 1", "1": "Dimension 2", "2": "Dimension 3"},
    template="plotly_white", width=1000, height=800, opacity=.5)

figure_3D.update_traces(marker_size=11)
figure_3D.update_layout(legend_itemsizing="constant", legend_font_size=10, font=dict(size=12))
figure_3D.show()